# 02 - Data Processing

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import numpy as np
import pandas as pd

import constants as cnst
import stock_utils as su

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
stock_symbols = su.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['HDFCBANK', 'ITBEES']

In [4]:
STOCK_SYMBOL = stock_symbols[0]
STOCK_SYMBOL

'HDFCBANK'

## Data loading

In [5]:
stock_df = pd.read_parquet(cnst.NSE_DATA_DIR.joinpath(STOCK_SYMBOL, "consolidated.parquet"))
stock_df = stock_df.drop(columns = ['series', 'PREV. CLOSE'])
stock_df.columns = [
    'Date',
    'Open', 'High', 'Low', 'LTP', 'Close',
    'VWAP', '52W H', '52W L',
    'Volume', 'Value', '#Trades'
]
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades
0,2020-01-01,1276.10,1280.00,1270.60,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625
1,2020-01-02,1279.00,1288.00,1279.00,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570
2,2020-01-03,1282.20,1285.00,1263.60,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066
3,2020-01-06,1260.00,1261.80,1236.00,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007
4,2020-01-07,1258.90,1271.45,1252.25,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026
...,...,...,...,...,...,...,...,...,...,...,...,...
1004,2023-12-22,1683.60,1685.90,1667.10,1668.70,1670.85,1675.74,1757.5,1460.25,24289425,4.070271e+10,450284
1005,2023-12-26,1673.25,1685.95,1668.55,1684.10,1682.45,1678.46,1757.5,1460.25,9022928,1.514458e+10,243596
1006,2023-12-27,1681.50,1706.50,1678.60,1702.10,1703.30,1695.87,1757.5,1460.25,13504539,2.290188e+10,321373
1007,2023-12-28,1709.30,1721.40,1702.00,1708.85,1705.25,1711.26,1757.5,1460.25,22038235,3.771305e+10,423062


In [6]:
stock_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     1009 non-null   datetime64[ns]
 1   Open     1009 non-null   float64       
 2   High     1009 non-null   float64       
 3   Low      1009 non-null   float64       
 4   LTP      1009 non-null   float64       
 5   Close    1009 non-null   float64       
 6   VWAP     1009 non-null   float64       
 7   52W H    1009 non-null   float64       
 8   52W L    1009 non-null   float64       
 9   Volume   1009 non-null   int64         
 10  Value    1009 non-null   float64       
 11  #Trades  1009 non-null   int64         
dtypes: datetime64[ns](1), float64(9), int64(2)
memory usage: 94.7 KB


## Feature engineering

### Daily candle based columns

In [7]:
stock_df['IsGreen'] = (stock_df['Close'] >= stock_df['Open']).astype(int)
stock_df['Is52WLow'] = np.isclose(stock_df['Low'], stock_df['52W L'], atol = 0, rtol = 1e-4).astype(int)
stock_df['Is52WHigh'] = np.isclose(stock_df['High'], stock_df['52W H'], atol = 0, rtol = 1e-4).astype(int)
stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,IsGreen,Is52WLow,Is52WHigh
75,2020-03-26,870.10,937.70,841.00,898.00,901.10,898.62,2503.3,738.75,28521681,2.563014e+10,627160,1,0,0
778,2023-01-23,1666.25,1682.60,1661.40,1671.90,1673.10,1674.88,1722.1,1271.60,7722200,1.293379e+10,302449,1,0,0
573,2022-03-29,1441.00,1460.25,1430.00,1458.00,1451.80,1442.08,1725.0,1292.00,6188724,8.924654e+09,203589,1,0,0
197,2020-09-22,1054.05,1058.15,1029.50,1038.00,1035.40,1042.49,1305.5,738.75,9310339,9.705908e+09,218683,0,0,0
127,2020-06-16,975.00,993.95,952.00,989.15,990.40,978.56,2503.3,738.75,22700113,2.221335e+10,318791,1,0,0
18,2020-01-27,1235.00,1235.00,1211.75,1213.50,1213.20,1220.97,2503.3,1084.00,9444276,1.153119e+10,268331,0,0,0
189,2020-09-10,1103.00,1104.85,1084.00,1091.95,1090.55,1095.11,2288.8,738.75,7594209,8.316494e+09,132365,0,0,0
650,2022-07-19,1337.00,1358.55,1337.00,1348.20,1348.05,1349.05,1725.0,1271.60,7896269,1.065245e+10,141741,1,0,0
993,2023-12-07,1625.00,1634.00,1616.20,1630.50,1630.45,1628.48,1757.5,1460.25,10225193,1.665151e+10,219921,1,0,0
338,2021-04-19,1390.00,1417.70,1372.30,1412.60,1412.40,1393.68,1641.0,826.10,12034621,1.677236e+10,325777,1,0,0


In [8]:
((stock_df[['IsGreen', 'Is52WLow', 'Is52WHigh']].sum() * 100) / len(stock_df)).round(2)

IsGreen      49.36
Is52WLow      1.49
Is52WHigh     2.58
dtype: float64

In [9]:
stock_df.shape

(1009, 15)

### Date based columns

In [10]:
stock_df['Day'] = stock_df['Date'].dt.day
stock_df['Month'] = stock_df['Date'].dt.month
stock_df['Year'] = stock_df['Date'].dt.year
stock_df['Weekday'] = stock_df['Date'].dt.weekday
stock_df['DayOfYear'] = stock_df['Date'].dt.day_of_year
stock_df['Quarter'] = stock_df['Date'].dt.quarter
stock_df['DaysSinceLastTradingSession'] = (stock_df['Date'] - stock_df['Date'].shift(1)).dt.days.fillna(1).astype(int)

stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession
203,2020-09-30,1060.0,1086.90,1052.00,1078.0,1078.60,1070.11,1305.5,738.75,9407038,1.006660e+10,226805,1,0,0,30,9,2020,2,274,3,1
908,2023-08-03,1640.0,1651.35,1623.00,1632.6,1628.65,1633.08,1757.5,1365.00,28836973,4.709311e+10,439722,0,0,0,3,8,2023,3,215,3,1
769,2023-01-10,1600.0,1600.00,1565.00,1570.0,1568.30,1572.13,1722.1,1271.60,8185883,1.286923e+10,290553,0,0,0,10,1,2023,1,10,1,1
8,2020-01-13,1282.7,1296.50,1276.00,1287.0,1286.00,1289.33,2503.3,1084.00,3725784,4.803756e+09,107121,1,0,0,13,1,2020,0,13,1,3
121,2020-06-08,1060.0,1066.65,1006.40,1016.8,1015.90,1031.68,2503.3,738.75,24906362,2.569546e+10,388541,0,0,0,8,6,2020,0,160,2,3
449,2021-09-28,1632.0,1632.00,1582.00,1606.8,1615.05,1605.07,1641.0,1042.65,6386895,1.025142e+10,193041,0,0,0,28,9,2021,1,271,3,1
20,2020-01-29,1225.3,1242.00,1222.25,1235.8,1235.85,1235.01,2503.3,1084.00,6894175,8.514372e+09,155174,1,0,0,29,1,2020,2,29,1,1
685,2022-09-09,1496.2,1509.00,1494.05,1498.0,1498.60,1501.98,1725.0,1271.60,7564444,1.136167e+10,159222,1,0,0,9,9,2022,4,252,3,1
247,2020-12-03,1429.0,1432.00,1374.05,1381.0,1377.20,1396.88,1464.4,738.75,18966611,2.649409e+10,370031,0,0,0,3,12,2020,3,338,4,1
538,2022-02-04,1528.4,1532.85,1514.10,1521.1,1524.00,1523.56,1725.0,1353.00,5789979,8.821363e+09,168452,0,0,0,4,2,2022,4,35,1,1


In [11]:
stock_df.shape

(1009, 22)

### Moving averages

#### `Close`

In [12]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Close {window}MA'] = stock_df['Close'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Close.*")

,Close,Close 3MA,Close 7MA,Close 15MA,Close 30MA
0,1278.60,1278.60,1278.60,1278.60,1278.60
1,1286.75,1282.68,1282.68,1282.68,1282.68
2,1268.40,1277.92,1277.92,1277.92,1277.92
3,1240.95,1265.37,1268.68,1268.68,1268.68
4,1260.60,1256.65,1267.06,1267.06,1267.06
...,...,...,...,...,...
1004,1670.85,1671.52,1661.41,1646.06,1582.67
1005,1682.45,1680.00,1666.02,1650.93,1589.04
1006,1703.30,1685.53,1672.70,1656.24,1595.81
1007,1705.25,1697.00,1679.78,1661.40,1603.03


#### `VWAP`

In [13]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'VWAP {window}MA'] = stock_df['VWAP'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "VWAP.*")

,VWAP,VWAP 3MA,VWAP 7MA,VWAP 15MA,VWAP 30MA
0,1276.64,1276.64,1276.64,1276.64,1276.64
1,1284.56,1280.60,1280.60,1280.60,1280.60
2,1270.48,1277.23,1277.23,1277.23,1277.23
3,1247.24,1267.43,1269.73,1269.73,1269.73
4,1261.48,1259.73,1268.08,1268.08,1268.08
...,...,...,...,...,...
1004,1675.74,1670.28,1660.89,1645.06,1581.69
1005,1678.46,1675.69,1664.81,1650.81,1588.08
1006,1695.87,1683.36,1670.60,1655.46,1594.63
1007,1711.26,1695.20,1678.24,1661.05,1601.98


#### `Volume`

In [14]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Volume {window}MA'] = stock_df['Volume'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Volume.*")

,Volume,Volume 3MA,Volume 7MA,Volume 15MA,Volume 30MA
0,1836849,1836849,1836849,1836849,1836849
1,3068583,2452716,2452716,2452716,2452716
2,5427775,3444402,3444402,3444402,3444402
3,5445093,4647150,3944575,3944575,3944575
4,7362247,6078371,4628109,4628109,4628109
...,...,...,...,...,...
1004,24289425,19900101,23959521,20674469,16842909
1005,9022928,17202324,22736199,19587249,16910087
1006,13504539,15605630,14748481,18746524,17294389
1007,22038235,14855234,16610032,18928638,17685565


#### `Value`

In [15]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Value {window}MA'] = stock_df['Value'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Value.*")

,Value,Value 3MA,Value 7MA,Value 15MA,Value 30MA
0,2.345001e+09,2345000988,2345000988,2345000988,2345000988
1,3.941792e+09,3143396262,3143396262,3143396262,3143396262
2,6.895886e+09,4394226092,4394226092,4394226092,4394226092
3,6.791348e+09,5876341707,4993506527,4993506527,4993506527
4,9.287302e+09,7658178376,5852265530,5852265530,5852265530
...,...,...,...,...,...
1004,4.070271e+10,33252816061,39779977233,34019236236,26928124028
1005,1.514458e+10,28817261735,37795582215,32339953679,27085677731
1006,2.290188e+10,26249724747,24651270134,31035709963,27750333500
1007,3.771305e+10,25253170191,27905665883,31455274908,28495473480


#### `#Trades`

In [16]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'#Trades {window}MA'] = stock_df['#Trades'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "#Trades.*")

,#Trades,#Trades 3MA,#Trades 7MA,#Trades 15MA,#Trades 30MA
0,46625,46625,46625,46625,46625
1,104570,75597,75597,75597,75597
2,157066,102753,102753,102753,102753
3,155007,138881,115817,115817,115817
4,189026,167033,130458,130458,130458
...,...,...,...,...,...
1004,450284,417161,361824,361372,308156
1005,243596,359931,347304,348166,310241
1006,321373,338417,333095,343117,318093
1007,423062,329343,356645,348588,321565


### Target columns

In [17]:
for window in cnst.TARGET_WINDOWS:
    stock_df[f'Target {window}D'] = stock_df['Close'].shift(-window)

stock_df.filter(regex = "Target.*")

,Target 3D,Target 7D,Target 15D,Target 30D
0,1240.95,1282.70,1240.85,1240.60
1,1260.60,1286.00,1244.85,1249.00
2,1257.30,1289.50,1244.55,1241.40
3,1271.40,1284.25,1213.20,1219.35
4,1282.70,1287.65,1223.20,1217.15
...,...,...,...,...
1004,1705.25,NaN,NaN,NaN
1005,1709.25,NaN,NaN,NaN
1006,NaN,NaN,NaN,NaN
1007,NaN,NaN,NaN,NaN


### Saving processed data

In [18]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-processed.parquet'), 
    index = False
)
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 3MA,Close 7MA,Close 15MA,Close 30MA,VWAP 3MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,Volume 3MA,Volume 7MA,Volume 15MA,Volume 30MA,Value 3MA,Value 7MA,Value 15MA,Value 30MA,#Trades 3MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,Target 3D,Target 7D,Target 15D,Target 30D
0,2020-01-01,1276.10,1280.00,1270.60,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,1,0,0,1,1,2020,2,1,1,1,1278.60,1278.60,1278.60,1278.60,1276.64,1276.64,1276.64,1276.64,1836849,1836849,1836849,1836849,2345000988,2345000988,2345000988,2345000988,46625,46625,46625,46625,1240.95,1282.70,1240.85,1240.60
1,2020-01-02,1279.00,1288.00,1279.00,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,1,0,0,2,1,2020,3,2,1,1,1282.68,1282.68,1282.68,1282.68,1280.60,1280.60,1280.60,1280.60,2452716,2452716,2452716,2452716,3143396262,3143396262,3143396262,3143396262,75597,75597,75597,75597,1260.60,1286.00,1244.85,1249.00
2,2020-01-03,1282.20,1285.00,1263.60,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,0,0,0,3,1,2020,4,3,1,1,1277.92,1277.92,1277.92,1277.92,1277.23,1277.23,1277.23,1277.23,3444402,3444402,3444402,3444402,4394226092,4394226092,4394226092,4394226092,102753,102753,102753,102753,1257.30,1289.50,1244.55,1241.40
3,2020-01-06,1260.00,1261.80,1236.00,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,0,0,0,6,1,2020,0,6,1,3,1265.37,1268.68,1268.68,1268.68,1267.43,1269.73,1269.73,1269.73,4647150,3944575,3944575,3944575,5876341707,4993506527,4993506527,4993506527,138881,115817,115817,115817,1271.40,1284.25,1213.20,1219.35
4,2020-01-07,1258.90,1271.45,1252.25,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,1,0,0,7,1,2020,1,7,1,1,1256.65,1267.06,1267.06,1267.06,1259.73,1268.08,1268.08,1268.08,6078371,4628109,4628109,4628109,7658178376,5852265530,5852265530,5852265530,167033,130458,130458,130458,1282.70,1287.65,1223.20,1217.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,2023-12-22,1683.60,1685.90,1667.10,1668.70,1670.85,1675.74,1757.5,1460.25,24289425,4.070271e+10,450284,0,0,0,22,12,2023,4,356,4,1,1671.52,1661.41,1646.06,1582.67,1670.28,1660.89,1645.06,1581.69,19900101,23959521,20674469,16842909,33252816061,39779977233,34019236236,26928124028,417161,361824,361372,308156,1705.25,NaN,NaN,NaN
1005,2023-12-26,1673.25,1685.95,1668.55,1684.10,1682.45,1678.46,1757.5,1460.25,9022928,1.514458e+10,243596,1,0,0,26,12,2023,1,360,4,4,1680.00,1666.02,1650.93,1589.04,1675.69,1664.81,1650.81,1588.08,17202324,22736199,19587249,16910087,28817261735,37795582215,32339953679,27085677731,359931,347304,348166,310241,1709.25,NaN,NaN,NaN
1006,2023-12-27,1681.50,1706.50,1678.60,1702.10,1703.30,1695.87,1757.5,1460.25,13504539,2.290188e+10,321373,1,0,0,27,12,2023,2,361,4,1,1685.53,1672.70,1656.24,1595.81,1683.36,1670.60,1655.46,1594.63,15605630,14748481,18746524,17294389,26249724747,24651270134,31035709963,27750333500,338417,333095,343117,318093,NaN,NaN,NaN,NaN
1007,2023-12-28,1709.30,1721.40,1702.00,1708.85,1705.25,1711.26,1757.5,1460.25,22038235,3.771305e+10,423062,0,0,0,28,12,2023,3,362,4,1,1697.00,1679.78,1661.40,1603.03,1695.20,1678.24,1661.05,1601.98,14855234,16610032,18928638,17685565,25253170191,27905665883,31455274908,28495473480,329343,356645,348588,321565,NaN,NaN,NaN,NaN


## Data standardization

### Stock price based columns

In [19]:
stock_price_cols = [
    'Open', 'High', 'Low', 'LTP', '52W H', '52W L'
] + (
    stock_df.filter(regex = "Close.*").columns.to_list() +
    stock_df.filter(regex = "VWAP.*").columns.to_list() +
    stock_df.filter(regex = "Target.*").columns.to_list()
)

stock_price_cols

['Open',
 'High',
 'Low',
 'LTP',
 '52W H',
 '52W L',
 'Close',
 'Close 3MA',
 'Close 7MA',
 'Close 15MA',
 'Close 30MA',
 'VWAP',
 'VWAP 3MA',
 'VWAP 7MA',
 'VWAP 15MA',
 'VWAP 30MA',
 'Target 3D',
 'Target 7D',
 'Target 15D',
 'Target 30D']

In [20]:
stock_df[stock_price_cols] = stock_df[stock_price_cols].div(stock_df['Close'], axis = 0).round(3)
stock_df[stock_price_cols]

,Open,High,Low,LTP,52W H,52W L,Close,Close 3MA,Close 7MA,Close 15MA,Close 30MA,VWAP,VWAP 3MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,1.958,0.848,1.0,1.000,1.000,1.000,1.000,0.998,0.998,0.998,0.998,0.998,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,1.945,0.842,1.0,0.997,0.997,0.997,0.997,0.998,0.995,0.995,0.995,0.995,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.974,0.855,1.0,1.008,1.008,1.008,1.008,1.002,1.007,1.007,1.007,1.007,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,2.017,0.874,1.0,1.020,1.022,1.022,1.022,1.005,1.021,1.023,1.023,1.023,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.986,0.860,1.0,0.997,1.005,1.005,1.005,1.001,0.999,1.006,1.006,1.006,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,1.008,1.009,0.998,0.999,1.052,0.874,1.0,1.000,0.994,0.985,0.947,1.003,1.000,0.994,0.985,0.947,1.021,NaN,NaN,NaN
1005,0.995,1.002,0.992,1.001,1.045,0.868,1.0,0.999,0.990,0.981,0.944,0.998,0.996,0.990,0.981,0.944,1.016,NaN,NaN,NaN
1006,0.987,1.002,0.985,0.999,1.032,0.857,1.0,0.990,0.982,0.972,0.937,0.996,0.988,0.981,0.972,0.936,NaN,NaN,NaN,NaN
1007,1.002,1.009,0.998,1.002,1.031,0.856,1.0,0.995,0.985,0.974,0.940,1.004,0.994,0.984,0.974,0.939,NaN,NaN,NaN,NaN


### Volume based columns

In [21]:
volume_cols = stock_df.filter(regex = "Volume.*").columns.to_list()

volume_cols

['Volume', 'Volume 3MA', 'Volume 7MA', 'Volume 15MA', 'Volume 30MA']

In [22]:
stock_df[volume_cols] = stock_df[volume_cols].div(stock_df['Volume'], axis = 0).round(3)
stock_df[volume_cols]

,Volume,Volume 3MA,Volume 7MA,Volume 15MA,Volume 30MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.799,0.799,0.799,0.799
2,1.0,0.635,0.635,0.635,0.635
3,1.0,0.853,0.724,0.724,0.724
4,1.0,0.826,0.629,0.629,0.629
...,...,...,...,...,...
1004,1.0,0.819,0.986,0.851,0.693
1005,1.0,1.907,2.520,2.171,1.874
1006,1.0,1.156,1.092,1.388,1.281
1007,1.0,0.674,0.754,0.859,0.802


### Value based columns

In [23]:
value_cols = stock_df.filter(regex = "Value.*").columns.to_list()

value_cols

['Value', 'Value 3MA', 'Value 7MA', 'Value 15MA', 'Value 30MA']

In [24]:
stock_df[value_cols] = stock_df[value_cols].div(stock_df['Value'], axis = 0).round(3)
stock_df[value_cols]

,Value,Value 3MA,Value 7MA,Value 15MA,Value 30MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.797,0.797,0.797,0.797
2,1.0,0.637,0.637,0.637,0.637
3,1.0,0.865,0.735,0.735,0.735
4,1.0,0.825,0.630,0.630,0.630
...,...,...,...,...,...
1004,1.0,0.817,0.977,0.836,0.662
1005,1.0,1.903,2.496,2.135,1.788
1006,1.0,1.146,1.076,1.355,1.212
1007,1.0,0.670,0.740,0.834,0.756


### Trade count based columns

In [25]:
trade_count_cols = stock_df.filter(regex = "#Trades.*").columns.to_list()

trade_count_cols

['#Trades', '#Trades 3MA', '#Trades 7MA', '#Trades 15MA', '#Trades 30MA']

In [26]:
stock_df[trade_count_cols] = stock_df[trade_count_cols].div(stock_df['#Trades'], axis = 0).round(3)
stock_df[trade_count_cols]

,#Trades,#Trades 3MA,#Trades 7MA,#Trades 15MA,#Trades 30MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.723,0.723,0.723,0.723
2,1.0,0.654,0.654,0.654,0.654
3,1.0,0.896,0.747,0.747,0.747
4,1.0,0.884,0.690,0.690,0.690
...,...,...,...,...,...
1004,1.0,0.926,0.804,0.803,0.684
1005,1.0,1.478,1.426,1.429,1.274
1006,1.0,1.053,1.036,1.068,0.990
1007,1.0,0.778,0.843,0.824,0.760


### Dropping unnecessary columns

In [27]:
stock_df = stock_df.drop(columns = ['Date', 'Close', 'Volume', 'Value', '#Trades'])
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 3MA,Close 7MA,Close 15MA,Close 30MA,VWAP 3MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,Volume 3MA,Volume 7MA,Volume 15MA,Volume 30MA,Value 3MA,Value 7MA,Value 15MA,Value 30MA,#Trades 3MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,0.998,1.958,0.848,1,0,0,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.998,0.998,0.998,0.998,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,0.998,1.945,0.842,1,0,0,2,1,2020,3,2,1,1,0.997,0.997,0.997,0.997,0.995,0.995,0.995,0.995,0.799,0.799,0.799,0.799,0.797,0.797,0.797,0.797,0.723,0.723,0.723,0.723,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.002,1.974,0.855,0,0,0,3,1,2020,4,3,1,1,1.008,1.008,1.008,1.008,1.007,1.007,1.007,1.007,0.635,0.635,0.635,0.635,0.637,0.637,0.637,0.637,0.654,0.654,0.654,0.654,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,1.005,2.017,0.874,0,0,0,6,1,2020,0,6,1,3,1.020,1.022,1.022,1.022,1.021,1.023,1.023,1.023,0.853,0.724,0.724,0.724,0.865,0.735,0.735,0.735,0.896,0.747,0.747,0.747,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.001,1.986,0.860,1,0,0,7,1,2020,1,7,1,1,0.997,1.005,1.005,1.005,0.999,1.006,1.006,1.006,0.826,0.629,0.629,0.629,0.825,0.630,0.630,0.630,0.884,0.690,0.690,0.690,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,1.008,1.009,0.998,0.999,1.003,1.052,0.874,0,0,0,22,12,2023,4,356,4,1,1.000,0.994,0.985,0.947,1.000,0.994,0.985,0.947,0.819,0.986,0.851,0.693,0.817,0.977,0.836,0.662,0.926,0.804,0.803,0.684,1.021,NaN,NaN,NaN
1005,0.995,1.002,0.992,1.001,0.998,1.045,0.868,1,0,0,26,12,2023,1,360,4,4,0.999,0.990,0.981,0.944,0.996,0.990,0.981,0.944,1.907,2.520,2.171,1.874,1.903,2.496,2.135,1.788,1.478,1.426,1.429,1.274,1.016,NaN,NaN,NaN
1006,0.987,1.002,0.985,0.999,0.996,1.032,0.857,1,0,0,27,12,2023,2,361,4,1,0.990,0.982,0.972,0.937,0.988,0.981,0.972,0.936,1.156,1.092,1.388,1.281,1.146,1.076,1.355,1.212,1.053,1.036,1.068,0.990,NaN,NaN,NaN,NaN
1007,1.002,1.009,0.998,1.002,1.004,1.031,0.856,0,0,0,28,12,2023,3,362,4,1,0.995,0.985,0.974,0.940,0.994,0.984,0.974,0.939,0.674,0.754,0.859,0.802,0.670,0.740,0.834,0.756,0.778,0.843,0.824,0.760,NaN,NaN,NaN,NaN


### Saving standardized data

In [28]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-standardized.parquet'), 
    index = False
)
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 3MA,Close 7MA,Close 15MA,Close 30MA,VWAP 3MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,Volume 3MA,Volume 7MA,Volume 15MA,Volume 30MA,Value 3MA,Value 7MA,Value 15MA,Value 30MA,#Trades 3MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,0.998,1.958,0.848,1,0,0,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.998,0.998,0.998,0.998,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,0.998,1.945,0.842,1,0,0,2,1,2020,3,2,1,1,0.997,0.997,0.997,0.997,0.995,0.995,0.995,0.995,0.799,0.799,0.799,0.799,0.797,0.797,0.797,0.797,0.723,0.723,0.723,0.723,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.002,1.974,0.855,0,0,0,3,1,2020,4,3,1,1,1.008,1.008,1.008,1.008,1.007,1.007,1.007,1.007,0.635,0.635,0.635,0.635,0.637,0.637,0.637,0.637,0.654,0.654,0.654,0.654,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,1.005,2.017,0.874,0,0,0,6,1,2020,0,6,1,3,1.020,1.022,1.022,1.022,1.021,1.023,1.023,1.023,0.853,0.724,0.724,0.724,0.865,0.735,0.735,0.735,0.896,0.747,0.747,0.747,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.001,1.986,0.860,1,0,0,7,1,2020,1,7,1,1,0.997,1.005,1.005,1.005,0.999,1.006,1.006,1.006,0.826,0.629,0.629,0.629,0.825,0.630,0.630,0.630,0.884,0.690,0.690,0.690,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,1.008,1.009,0.998,0.999,1.003,1.052,0.874,0,0,0,22,12,2023,4,356,4,1,1.000,0.994,0.985,0.947,1.000,0.994,0.985,0.947,0.819,0.986,0.851,0.693,0.817,0.977,0.836,0.662,0.926,0.804,0.803,0.684,1.021,NaN,NaN,NaN
1005,0.995,1.002,0.992,1.001,0.998,1.045,0.868,1,0,0,26,12,2023,1,360,4,4,0.999,0.990,0.981,0.944,0.996,0.990,0.981,0.944,1.907,2.520,2.171,1.874,1.903,2.496,2.135,1.788,1.478,1.426,1.429,1.274,1.016,NaN,NaN,NaN
1006,0.987,1.002,0.985,0.999,0.996,1.032,0.857,1,0,0,27,12,2023,2,361,4,1,0.990,0.982,0.972,0.937,0.988,0.981,0.972,0.936,1.156,1.092,1.388,1.281,1.146,1.076,1.355,1.212,1.053,1.036,1.068,0.990,NaN,NaN,NaN,NaN
1007,1.002,1.009,0.998,1.002,1.004,1.031,0.856,0,0,0,28,12,2023,3,362,4,1,0.995,0.985,0.974,0.940,0.994,0.984,0.974,0.939,0.674,0.754,0.859,0.802,0.670,0.740,0.834,0.756,0.778,0.843,0.824,0.760,NaN,NaN,NaN,NaN
